In [ ]:
from pyspark.sql import SparkSession

In [ ]:
# Initialize Spark session
spark = SparkSession.builder.appName("Shuffle Example").getOrCreate()

In [ ]:
# Create a DataFrame with sample data
data = [
    (1, "Alice", 100),
    (2, "Bob", 200),
    (3, "Alice", 150),
    (4, "Bob", 250),
    (5, "Charlie", 300)
]

columns = ["id", "name", "amount"]
df = spark.createDataFrame(data, columns)

In [ ]:
# Group by "name" and sum the "amount" to force a shuffle
grouped_df = df.groupBy("name").sum("amount")

In [ ]:
grouped_df.explain(mode="extended")  # Use "extended" to get detailed plan

In [ ]:
# Trigger execution (e.g., show results)
grouped_df.show()

In [ ]:
#Stop Spark Session
spark.stop()